In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import datetime

In [2]:
americanas_baseurl = "https://www.americanas.com.br"

In [3]:
# get categories from sitemap

AMERICANAS_BASEURL = "https://www.americanas.com.br"

def categoriesFromSitemap(sitemap_url):
    soup = BeautifulSoup(requests.get(sitemap_url).text, "html5lib")

    categories={}

    for cat in soup.find_all("li", {"class":"child-level-1"}):
        cat_a = cat.find("a")
        cat_name, cat_url = (cat_a.text, cat_a['href'])
        categories[cat_name] = {'url':cat_url, 'subcategories':{}}
        for subcat in cat.find_all("li",{"class":"child-level-2"}):
            subcat_a = subcat.find("a")
            subcat_name, subcat_url = (subcat_a.text, subcat_a['href'])
            categories[cat_name]['subcategories'][subcat_name] = subcat_url
    
    return categories


def getSubcategoriesUrls(categoriesDict):
    return [ url for cat,subcats in categoriesDict.items() for subcat,url in subcats['subcategories'].items() ]

In [4]:
def getProductData(productsPage_url, ordering='BestSellers', limit_num_offsets=None):
    print("Retrieving products from "+productsPage_url)
    productsPage = BeautifulSoup( requests.get(AMERICANAS_BASEURL+"/"+productsPage_url).text,"html5lib" )    

    products = []
    num_products = int(productsPage.find("aside",{'class':"sortbar"}).find("span").text.split()[0].replace('.',''))
    
     # these are url params that should be used for loading items pages
    limit = 24 # This is apparently hardcoded
    
    num_offsets = num_products//limit if num_products%limit==0 else (num_products//limit)+1
    
    if limit_num_offsets is not None: 
        num_offsets = min(limit_num_offsets,num_offsets)
    
    # Collects all products moving through offsets
    for offset in range(num_offsets):     

        print("  Collecting items from offset {} of {}".format(offset+1, num_offsets))

        requestUrl = AMERICANAS_BASEURL+"/"+productsPage_url+"?limite="+str(limit)+\
                                                             "&offset="+str(offset*limit)+\
                                                             "&ordenacao="+ordering
                
        productsGrid = BeautifulSoup( requests.get(requestUrl).text, "html5lib" )
        print("  ",format(requestUrl))        
        
        try:
            productItems = productsGrid.find("div",{'data-component':'productgrid'}).find_all("div",{'class':'product-grid-item'})
            # Retrieve data
            for i in productItems:
                try:
                    item = i.find("a")
                    item_title, item_code = (item['title'],item['href'].split("?")[0].split("/")[-1])
                    item_price = ' '.join( i.text for i in item.find_all("span",{'class':"value"}) )

                    try: item_discount = i.find("span",{'class':"label-discount-rate"}).text
                    except: item_discount=None

                    try: item_previous_price = item.find("del",{'class':"card-product-price-from-value"}).text
                    except: item_previous_price = None

                    try: item_rating = item.find("span",{'class':"rating-star-average"}).text
                    except: item_rating = None

                    try: item_num_ratings = item.find("div",{'class':"rating-star-counter"}).text[1]
                    except: item_num_ratings = None


                    products += [(item_code,
                                  item_title,
                                  item_price,
                                  item_previous_price, 
                                  item_discount,
                                  item_rating,
                                  item_num_ratings,
                                  productsPage_url,
                                  datetime.datetime.utcnow())]
                except:
                    pass

                
        except AttributeError:
            print("Offset ended before expected:",offset)
            break
            
    print("Done")        
    return products

In [5]:
import csv

def writeData(fpath, data):
    with open(fpath,'a') as fout:
        csv_out=csv.writer(fout)
        for row in data:
            csv_out.writerow(row)

### Crawling for black friday promos

In [112]:
promos = [
    "hotsite/blackfriday-17/ate80",
    "hotsite/blackfriday-17/ate60",
    "hotsite/blackfriday-17/ate30",
    "hotsite/blackfriday-17/ate10"
]

In [126]:
!rm descs.csv prices.csv ratings.csv

In [127]:
for promo in promos:
    
    prods = getProductData(promo)
    
    writeData('descs.csv',[ (p[0],p[1],p[8]) for p in prods ])
    writeData('prices.csv',[ (p[0],p[2],p[3],p[4],p[7],p[8]) for p in prods ])
    writeData('ratings.csv',[ (p[0],p[5],p[6]) for p in prods ])

Retrieving products from hotsite/blackfriday-17/ate80
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=0&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=24&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=48&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=72&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=96&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=120&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=144&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=168&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=192&ordenacao=BestSellers
   http

   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1344&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1368&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1392&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1416&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1440&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1464&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1488&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1512&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate80?limite=24&offset=1536&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/black

   https://www.americanas.com.br/hotsite/blackfriday-17/ate30?limite=24&offset=24&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate30?limite=24&offset=48&ordenacao=BestSellers
Offset ended before expected: 2
Done
Retrieving products from hotsite/blackfriday-17/ate10
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=0&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=24&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=48&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=72&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=96&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offset=120&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/blackfriday-17/ate10?limite=24&offse

In [298]:
# Read data
descs = pd.read_csv('./sunday_descs.csv',header=None, 
                   names=['Product Id','Title', 'Datetime Crawled'])

prices = pd.read_csv('./sunday_prices.csv',header=None,
                     names=['Product Id','Price','Previous Price', 'Discount','Promo','Datetime Crawled'])

prices.set_index('Product Id', inplace=True)
descs.set_index('Product Id', inplace=True)

# Get percentage discount
prices['Discount'] = prices['Discount'].apply( lambda x: x if x is np.NaN else float(x.split("%")[0])/100 )

# Reformat prices data
getPrice = lambda x: x if x is np.NaN else float(x.split("R$")[1].replace('.','').replace(',','.'))
prices['Price'] = prices['Price'].apply(getPrice)
prices['Previous Price'] = prices['Previous Price'].apply(getPrice)

prices['Previous Price Inferred'] = prices['Price']/(1-prices['Discount'])

# Join tables
prods = descs[['Title']].join(prices)

In [299]:
prods['Product Id'] = prods.index

In [300]:
prods.drop_duplicates(subset=['Product Id','Title','Price','Discount'],inplace=True)

In [258]:
prods.index.is_unique

False

In [301]:
prods[prods.index.duplicated()]

,Title,Price,Previous Price,Discount,Promo,Datetime Crawled,Previous Price Inferred,Product Id
Product Id,,,,,,,,
114005310,Livro - Manual de Instrumentação Cirúrgica,18.0,NaN,0.82,hotsite/blackfriday-17/ate80,2017-11-26 14:49:56.836041,100.0,114005310


In [302]:
prods.loc[114005310]

,Title,Price,Previous Price,Discount,Promo,Datetime Crawled,Previous Price Inferred,Product Id
Product Id,,,,,,,,
114005310,Livro - Manual de Instrumentação Cirúrgica,23.99,103.0,0.76,hotsite/blackfriday-17/ate80,2017-11-26 14:49:27.375357,99.958333,114005310
114005310,Livro - Manual de Instrumentação Cirúrgica,18.00,NaN,0.82,hotsite/blackfriday-17/ate80,2017-11-26 14:49:56.836041,100.000000,114005310


### Crawling for Cyber Monday promos

In [312]:
promos = [
    "hotsite/cyber/ate80",
    "hotsite/cyber/ate60",
    "hotsite/cyber/ate30",
    "hotsite/cyber/ate10"
]

for promo in promos:
    
    prods = getProductData(promo,limit_num_offsets=100)
    
    writeData('descs.csv',[ (p[0],p[1],p[8]) for p in prods ])
    writeData('prices.csv',[ (p[0],p[2],p[3],p[4],p[7],p[8]) for p in prods ])
    writeData('ratings.csv',[ (p[0],p[5],p[6]) for p in prods ])

Retrieving products from hotsite/cyber/ate80
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=0&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=24&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=48&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=72&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=96&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=120&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=144&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=168&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=192&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate80?limite=24&offset=216&ordenacao=BestSellers
 

   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=96&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=120&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=144&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=168&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=192&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=216&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate30?limite=24&offset=240&ordenacao=BestSellers
Offset ended before expected: 10
Done
Retrieving products from hotsite/cyber/ate10
   https://www.americanas.com.br/hotsite/cyber/ate10?limite=24&offset=0&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate10?limite=24&offset=24&ordenacao=BestSellers
   https://www.americanas.com.br/hotsite/cyber/ate10?limi

---

## Working with the scraped data

In [6]:
descs_cyber = pd.read_csv('cyber_descs.csv',header=None,
                    names=['Product Id','Title', 'Datetime Crawled'])

prices_cyber = pd.read_csv('cyber_prices.csv',header=None,
                    names=['Product Id','Price','Previous Price', 'Discount','Promo','Datetime Crawled'])

descs_bf = pd.read_csv('./sunday_descs.csv',header=None, 
                   names=['Product Id','Title', 'Datetime Crawled'])

prices_bf = pd.read_csv('./sunday_prices.csv',header=None,
                     names=['Product Id','Price','Previous Price', 'Discount','Promo','Datetime Crawled'])

prices_cyber.set_index('Product Id', inplace=True, drop=False)
descs_cyber.set_index('Product Id', inplace=True, drop=False)
prices_bf.set_index('Product Id', inplace=True, drop=False)
descs_bf.set_index('Product Id', inplace=True, drop=False)

In [7]:
# Concatenate products tables and drop duplicates
prod_descs = pd.concat([descs_bf,descs_cyber]).drop_duplicates(subset=['Product Id','Title'])

In [8]:
prod_descs.shape

(3124, 3)

In [9]:
prod_descs.head()

,Product Id,Title,Datetime Crawled
Product Id,,,
128442334,128442334,Passadeira Compacta à Vapor Cadence Lisser Pro...,2017-11-26 14:49:20.281974
25660057,25660057,Furadeira impacto 1/2 Songhe Tools 550W 110V m...,2017-11-26 14:49:20.283970
20813260,20813260,Computador Desktop EasyPC Intel Core i5 8GB HD...,2017-11-26 14:49:20.285879
16064504,16064504,Relógio De Pulso Invicta 0074 Pro Diver Chrono...,2017-11-26 14:49:20.287835
11234730,11234730,Dvr Stand Alone Tríbrido Hdcvi Intelbras Hdcvi...,2017-11-26 14:49:20.289721


In [10]:
df1 = prod_descs[['Title']].join(prices_bf,how='right')
df2 = prod_descs[['Title']].join(prices_cyber,how='right')

prod = pd.concat([df1,df2])

# Get percentage discount
prod['Discount'] = prod['Discount'].apply( lambda x: x if x is np.NaN else float(x.split("%")[0])/100 )

# Reformat prices data
getPrice = lambda x: x if x is np.NaN else float(x.split("R$")[1].replace('.','').replace(',','.'))
prod['Price'] = prod['Price'].apply(getPrice)
prod['Previous Price'] = prod['Previous Price'].apply(getPrice)

prod['Previous Price Inferred'] = prod['Price']/(1-prod['Discount'])

In [11]:
prod.drop_duplicates(subset=['Price','Previous Price','Discount','Promo'],inplace=True)

In [12]:
prod.shape

(3830, 8)

In [13]:
prod.head()

,Title,Product Id,Price,Previous Price,Discount,Promo,Datetime Crawled,Previous Price Inferred
Product Id,,,,,,,,
148534,Frigobar c/ Porta de Vidro BZA08 76L Preto - B...,148534,1209.00,NaN,0.05,hotsite/blackfriday-17/ate10,2017-11-26 14:51:53.661827,1272.631579
167862,Livro - Faça Dar Certo,167862,4.38,NaN,0.77,hotsite/blackfriday-17/ate80,2017-11-26 14:49:38.177972,19.043478
198697,Massagem da Cabeça,198697,20.99,NaN,0.60,hotsite/blackfriday-17/ate80,2017-11-26 14:49:38.181962,52.475000
228073,DVD Solaris,228073,4.59,13.9,0.66,hotsite/blackfriday-17/ate80,2017-11-26 14:49:45.720859,13.500000
228139,Box DVD Coleção Friends: 3ª Temporada (4 DVDs),228139,9.99,NaN,0.60,hotsite/blackfriday-17/ate80,2017-11-26 14:49:22.249850,24.975000


In [14]:
prod.sort_values('Discount',ascending=False)

,Title,Product Id,Price,Previous Price,Discount,Promo,Datetime Crawled,Previous Price Inferred
Product Id,,,,,,,,
114005310,Livro - Manual de Instrumentação Cirúrgica,114005310,18.00,NaN,0.82,hotsite/blackfriday-17/ate80,2017-11-26 14:49:56.836041,100.000000
7403186,Conjunto Tramontina Design Collection Aluminio...,7403186,1499.90,7499.00,0.80,hotsite/cyber/ate80,2017-11-26 22:32:27.387220,7499.500000
7403186,Conjunto Tramontina Design Collection Aluminio...,7403186,1499.90,7499.00,0.80,hotsite/blackfriday-17/ate80,2017-11-26 14:49:50.042989,7499.500000
10565206,Gears Of War: Ultimate Edition - Xbox One,10565206,29.90,142.90,0.79,hotsite/blackfriday-17/ate80,2017-11-26 14:50:01.660415,142.380952
348301,DVD Exitz: A Única Saída,348301,4.99,24.90,0.79,hotsite/cyber/ate80,2017-11-26 22:32:30.776665,23.761905
17741229,Capa Folio E Suporte Para Samsung Galaxy Tab2 ...,17741229,1.99,9.89,0.79,hotsite/cyber/ate80,2017-11-26 22:31:34.333164,9.476190
121765587,Jogo Roubado,121765587,6.84,NaN,0.79,hotsite/cyber/ate80,2017-11-26 22:32:13.321804,32.571429
109918712,Livro - Ambiguidade,109918712,1.71,NaN,0.79,hotsite/cyber/ate80,2017-11-26 22:31:36.525405,8.142857
132610388,Livro - Meus Últimos Anos: Os Escritos Da Matu...,132610388,7.13,NaN,0.79,hotsite/blackfriday-17/ate80,2017-11-26 14:49:23.029796,33.952381


Now I have a set of promotional products which could be further monitored:

In [15]:
prod_descs[['Title','Product Id']].drop_duplicates()

,Title,Product Id
Product Id,,
128442334,Passadeira Compacta à Vapor Cadence Lisser Pro...,128442334
25660057,Furadeira impacto 1/2 Songhe Tools 550W 110V m...,25660057
20813260,Computador Desktop EasyPC Intel Core i5 8GB HD...,20813260
16064504,Relógio De Pulso Invicta 0074 Pro Diver Chrono...,16064504
11234730,Dvr Stand Alone Tríbrido Hdcvi Intelbras Hdcvi...,11234730
119847161,Bicicleta Mountain Bike Ferarri MTB Fibra de C...,119847161
22793439,Camisa Topper Botafogo Away 2016 Sn,22793439
20813400,Computador Desktop EasyPC Intel Core i5 4GB HD...,20813400
28765156,Micro-ondas Philco PMM24 21 Litros,28765156


#### Retrieving products categories

Let's write a function for crawling info for product ids:

In [64]:
def getProductInfo(prodId, info):
    requestUrl = AMERICANAS_BASEURL+"/produto/"+prodId
    productPage = BeautifulSoup( requests.get(requestUrl).text,"html5lib" )    
    
    vals = [prodId]
    for expr in info:
        try: vals += [expr[0](productPage)]
        except: vals += [expr[1]]
    vals += [datetime.datetime.utcnow()]
    
    return tuple(vals)
        
    
def getProductsInfo( productsIds, info, limit=None ):
    print("Retrieving products info...")
    productsInfoRecords = []
    productsFailed = []
    num_prods = prod_descs.shape[0]

    if (limit is not None) and (limit<num_prods):
        productsIds = productsIds[:limit]
        
    for i,prodId in enumerate( productsIds ):
        print("  ({}/{}) Crawling product".format(i+1,num_prods),prodId)
        try:
            productsInfoRecords += [getProductInfo(str(prodId),info)]
        except KeyboardInterrupt:
            raise(KeyboardInterrupt)
        except:
            print("  Failed retrieving info for",prodId) 
            productsFailed += [prodId]

    print("Done")   
    return (productsInfoRecords,productsFailed)

In [63]:
# info is organized as a tuple (expression,default)
info = [
    (lambda x: x.find("p",{'class':"sales-price"}).text, None), # promotional (or current) price
    (lambda x: x.find("strike",{'class':"regular-price"}).text,None), # previous (or regular) price
    (lambda x: '|'.join( i.text for i in x.find_all("span",{'class':"breadcrumb-title"}) ),None), # categories
    (lambda x: x.find("span",{'itemprop':"ratingValue"})['content'],None), # rating(num of stars)
    (lambda x: x.find("div",{'class':"rating-star-counter"})['content'],0) # num of ratings
]

In [ ]:
pinfo = getProductsInfo(prod_descs['Product Id'],info, limit=10)